### Обучение матриц проекции

Реализуем процесс обучения матриц проекции, который был предложен на SemEval CRIM. Для этого сначала требуется загрузить какие-нибудь эмбеддинги для слов и подобрать то, каких именно кандидатов требуется ранжировать при выдаче гиперонимов.

Начнём с простой стратегии. Возьмём обученную модель FastText и загрузим эмбеддинги из неё. После этого будем для многословных термов усреднять эмбеддинги.

In [1]:
import sys
import json
from os.path import join
import os
sys.path.append("../")
import fasttext as ft
from thesaurus_parsing.thesaurus_parser import ThesaurusParser
from collections import Counter
from tqdm import tqdm_notebook as tqdm

In [2]:
deeppavlov_embeddings = ft.load_model('../data/models/fasttext_deeppavlov.bin')

In [3]:
len(deeppavlov_embeddings.get_words())

1572343

Мы видим, что здесь есть векторы для полутора миллионов слов. Конечно же, использовать их все, как кандидаты в гиперонимы, было бы хорошо. Но тем не менее, поскольку метод в основном похож на kNN, это будет очень долго.

Вследствие этого, необходимо, кроме сущностей тезауруса, оставить лишь некоторый топ в качестве кандидатов в гиперонимы. Этот топ можно подобрать по tf-idf. Но для начала надо построить векторы для сущностей из тезауруса.

In [4]:
thesaurus = ThesaurusParser("../data/RuThes", need_closure=False)

In [5]:
vocab_embeddings = dict()

In [6]:
for _, entry_dict in thesaurus.text_entries.items():
    lemma = entry_dict['lemma']
    vocab_embeddings[lemma] = deeppavlov_embeddings.get_sentence_vector(lemma)

In [7]:
len(vocab_embeddings)

110176

Давайте пройдём по всем текстам, которые загрузились на данный момент, для слов, которые есть в словаре, посчитаем частоту слов

In [8]:
DIR_PATH = "/home/loginov-ra/MIPT/HypernymyDetection/data/Lenta/texts_tagged_processed_tree"
file_list = os.listdir(DIR_PATH)
file_list = [join(DIR_PATH, filename) for filename in file_list]

In [9]:
word_ctr = Counter()
no_deeppavlov = 0

for filename in tqdm(file_list):
    with open(filename, encoding='utf-8') as sentences_file:
        sentences = json.load(sentences_file)
        for sent in sentences:
            if 'deeppavlov' not in sent:
                no_deeppavlov += 1
                continue
            
            multitokens, _ = sent['multi']
            for t in multitokens:
                word_ctr[t] += 1

In [10]:
print(no_deeppavlov)

321


In [11]:
word_ctr.most_common(n=28)

[(',', 220492),
 ('.', 187998),
 ('в', 131132),
 ('"', 125326),
 ('на', 53704),
 ('и', 52685),
 ('-', 47165),
 ('с', 33065),
 ('что', 30618),
 ('быть', 30246),
 ('по', 30037),
 ('год', 24540),
 ('о', 22173),
 ('не', 20218),
 (')', 19771),
 ('(', 19535),
 ('который', 17922),
 ('он', 17243),
 (':', 16031),
 ('из', 15889),
 ('это', 15832),
 ('тот', 13658),
 ('за', 12046),
 ('как', 12001),
 ('один', 11591),
 ('--', 10985),
 ('к', 10023),
 ('сообщать', 9841)]

Видим, что для первых $27$ слов нет необходимости искать гиперонимы, для остальных уже может быть. Поэтому возьмёи пока первые $100000$ слов для работы с ними, посчитаем их эмбеддинги и добавим в словарь.

In [12]:
additional_words = word_ctr.most_common(n=100000)[27:]

In [13]:
for word, _ in additional_words:
    vocab_embeddings[word] = deeppavlov_embeddings.get_word_vector(word)

Удалим модель за ненадобностью

In [17]:
del deeppavlov_embeddings

NameError: name 'deeppavlov_embeddings' is not defined

In [18]:
len(vocab_embeddings)

171691

Видим, что было пересечение, и добавилась где-то 61000 слов
_________________

**Определение модели**

Определим модель, в которой будет 5 матриц проекции и логистическая регрессия на косинусных расстояниях до проекций.

In [89]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
device

'cpu'

In [102]:
class CRIMModel(nn.Module):
    def __init__(self, n_matrices=5, embedding_dim=300, init_sigma=0.01):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.n_matrices = n_matrices
        self.init_sigma = init_sigma
        
        matrix_shape = (n_matrices, 1, embedding_dim, embedding_dim)
        self.matrices = torch.FloatTensor(size=matrix_shape, device=device)
        self.prob_layer = nn.Linear(in_features=n_matrices, out_features=1)
        
        for i in range(n_matrices):
            eye_tensor = torch.FloatTensor(size=(embedding_dim, embedding_dim), device=device)
            noise_tensor = torch.FloatTensor(size=(embedding_dim, embedding_dim), device=device)
            torch.nn.init.eye_(eye_tensor)
            torch.nn.init.normal_(noise_tensor, std=init_sigma)
            self.matrices[i][0] = eye_tensor + noise_tensor
            
        torch.nn.init.normal_(self.prob_layer.weight, std=0.1)
        self.matrices = self.matrices.requires_grad_()
        
    def forward(self, input_dict):
        candidate = input_dict['candidate'].view((1, 1, self.embedding_dim))
        batch = input_dict['batch'].unsqueeze(-1)
        batch_size = batch.shape[0]
        projections = torch.matmul(self.matrices, batch).permute(1, 0, 2, 3).squeeze(-1)
        similarities = F.cosine_similarity(projections, candidate, dim=-1)
        logits = self.prob_layer(similarities)
        probas = torch.sigmoid(logits)
        return probas

In [103]:
model = CRIMModel()

In [104]:
args = {
    'batch': torch.randn(64, 300),
    'candidate': torch.randn(300)
}

In [105]:
probas = model(args)

In [106]:
probas

tensor([[0.3910],
        [0.3918],
        [0.3910],
        [0.3916],
        [0.3920],
        [0.3912],
        [0.3915],
        [0.3908],
        [0.3912],
        [0.3912],
        [0.3918],
        [0.3919],
        [0.3916],
        [0.3912],
        [0.3913],
        [0.3913],
        [0.3909],
        [0.3911],
        [0.3912],
        [0.3912],
        [0.3916],
        [0.3910],
        [0.3912],
        [0.3909],
        [0.3907],
        [0.3908],
        [0.3911],
        [0.3914],
        [0.3916],
        [0.3920],
        [0.3909],
        [0.3916],
        [0.3914],
        [0.3919],
        [0.3919],
        [0.3915],
        [0.3907],
        [0.3917],
        [0.3910],
        [0.3914],
        [0.3918],
        [0.3912],
        [0.3906],
        [0.3908],
        [0.3915],
        [0.3911],
        [0.3921],
        [0.3909],
        [0.3914],
        [0.3918],
        [0.3910],
        [0.3919],
        [0.3916],
        [0.3914],
        [0.3912],
        [0

In [118]:
optimizer = Adam(model.parameters(), lr=1)

In [119]:
while True:
    probas = model(args)
    loss = probas.sum()
    print('Loss:', loss.detach().numpy())
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Loss: 1.0959834e-12
Loss: 1.0959793e-12
Loss: 1.0959751e-12
Loss: 1.0959667e-12
Loss: 1.0959584e-12
Loss: 1.09595e-12
Loss: 1.0959416e-12
Loss: 1.0959332e-12
Loss: 1.0959208e-12
Loss: 1.0959082e-12
Loss: 1.0958873e-12
Loss: 1.0958748e-12
Loss: 1.095858e-12
Loss: 1.0958413e-12
Loss: 1.0958246e-12
Loss: 1.0958079e-12
Loss: 1.0957828e-12
Loss: 1.0957661e-12
Loss: 1.0957452e-12
Loss: 1.0957201e-12


KeyboardInterrupt: 

Во всяком случае, на текущий момент возможно переобучить модель под нужные значения